## 1. Setup e Importações

In [ ]:
# Importações básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# MLFlow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc, roc_auc_score,
    ConfusionMatrixDisplay
)

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Importações concluídas")
print(f"MLFlow version: {mlflow.__version__}")

## 2. Carregar e Explorar os Dados

In [ ]:
# Carregar dados
data_path = Path('../data/customer_churn.csv')

if not data_path.exists():
    print("⚠️ Dados não encontrados. Execute: python data/generate_data.py")
else:
    df = pd.read_csv(data_path)
    print("✅ Dados carregados com sucesso!")
    print(f"\nShape: {df.shape}")
    print(f"\nColunas: {list(df.columns)}")

In [ ]:
# Visualizar primeiras linhas
df.head()

In [ ]:
# Estatísticas descritivas
df.describe()

In [ ]:
# Distribuição da variável target
churn_counts = df['churn'].value_counts()
churn_pct = df['churn'].value_counts(normalize=True) * 100

print("Distribuição de Churn:")
print(f"  Sem churn (0): {churn_counts[0]} ({churn_pct[0]:.1f}%)")
print(f"  Com churn (1): {churn_counts[1]} ({churn_pct[1]:.1f}%)")

# Gráfico
fig, ax = plt.subplots(figsize=(8, 5))
churn_counts.plot(kind='bar', ax=ax, color=['green', 'red'])
ax.set_xlabel('Churn')
ax.set_ylabel('Quantidade')
ax.set_title('Distribuição de Churn')
ax.set_xticklabels(['Sem Churn', 'Com Churn'], rotation=0)
plt.tight_layout()
plt.show()

## 3. Preparar Dados

In [ ]:
# Separar features e target
X = df.drop(['cliente_id', 'churn'], axis=1)
y = df['churn']

print(f"Features: {list(X.columns)}")
print(f"\nShape de X: {X.shape}")
print(f"Shape de y: {y.shape}")

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Treino: {len(X_train)} amostras")
print(f"Teste: {len(X_test)} amostras")
print(f"\nDistribuição de churn no treino: {y_train.value_counts(normalize=True).to_dict()}")
print(f"Distribuição de churn no teste: {y_test.value_counts(normalize=True).to_dict()}")

---

## 4. Exemplo 1: Tracking Básico com MLFlow

Vamos treinar um modelo de Regressão Logística e usar o MLFlow para rastrear:
- Parâmetros do modelo
- Métricas de desempenho
- O modelo treinado

In [ ]:
# Configurar experimento
experiment_name = "Notebook_Churn_Prediction"
mlflow.set_experiment(experiment_name)

print(f"✅ Experimento configurado: {experiment_name}")

In [ ]:
# Treinar modelo com tracking
with mlflow.start_run(run_name="logistic_regression_baseline"):
    
    # Parâmetros
    params = {
        'C': 1.0,
        'solver': 'lbfgs',
        'max_iter': 100,
        'random_state': 42
    }
    
    print("🚀 Treinando Regressão Logística...")
    
    # Treinar
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    
    # Predições
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    # Métricas
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_proba)
    }
    
    # Logar no MLFlow
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.log_param("train_samples", len(X_train))
    mlflow.log_param("test_samples", len(X_test))
    
    # Salvar modelo
    mlflow.sklearn.log_model(model, "model")
    
    # Tags
    mlflow.set_tag("model_type", "Logistic Regression")
    mlflow.set_tag("notebook", "tutorial")
    
    print("\n✅ Treinamento concluído!")
    print("\n📊 Métricas:")
    for name, value in metrics.items():
        print(f"   {name}: {value:.4f}")
    
    run_id = mlflow.active_run().info.run_id
    print(f"\n🔗 Run ID: {run_id}")

💡 **Dica**: Abra o MLFlow UI em outro terminal com `mlflow ui` e acesse `http://localhost:5000` para ver os resultados!

---

## 5. Exemplo 2: Logging de Artefatos

Vamos adicionar visualizações e salvá-las como artefatos.

In [ ]:
with mlflow.start_run(run_name="logistic_regression_with_artifacts"):
    
    # Treinar modelo
    params = {'C': 1.0, 'solver': 'lbfgs', 'max_iter': 100, 'random_state': 42}
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    
    # Predições
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    # Métricas
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_proba)
    }
    
    # Logar parâmetros e métricas
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    
    # === ARTEFATO 1: Confusion Matrix ===
    fig, ax = plt.subplots(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel('Predito')
    ax.set_ylabel('Real')
    ax.set_title('Confusion Matrix')
    mlflow.log_figure(fig, "confusion_matrix.png")
    plt.show()
    plt.close()
    
    # === ARTEFATO 2: ROC Curve ===
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.2f})')
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('Taxa de Falsos Positivos')
    ax.set_ylabel('Taxa de Verdadeiros Positivos')
    ax.set_title('Curva ROC')
    ax.legend(loc="lower right")
    ax.grid(alpha=0.3)
    mlflow.log_figure(fig, "roc_curve.png")
    plt.show()
    plt.close()
    
    # === ARTEFATO 3: Feature Importance ===
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': np.abs(model.coef_[0])
    }).sort_values('importance', ascending=True)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.barh(feature_importance['feature'], feature_importance['importance'])
    ax.set_xlabel('Importância (|coeficiente|)')
    ax.set_title('Feature Importance')
    mlflow.log_figure(fig, "feature_importance.png")
    plt.tight_layout()
    plt.show()
    plt.close()
    
    # Salvar modelo
    mlflow.sklearn.log_model(model, "model")
    
    print("\n✅ Modelo treinado e artefatos salvos!")
    print(f"   - Confusion Matrix")
    print(f"   - ROC Curve")
    print(f"   - Feature Importance")

---

## 6. Exemplo 3: Comparação de Modelos

Vamos treinar 3 modelos diferentes e comparar os resultados.

In [ ]:
# Função para treinar e avaliar modelo
def train_and_evaluate(model, model_name, params):
    """Treina modelo e loga no MLFlow."""
    
    with mlflow.start_run(run_name=model_name):
        # Treinar
        model.fit(X_train, y_train)
        
        # Predições
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
        
        # Métricas
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_proba)
        }
        
        # Logar
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        mlflow.set_tag("model_type", model_name)
        mlflow.sklearn.log_model(model, "model")
        
        return metrics

print("Função de treinamento definida ✅")

In [ ]:
# Modelo 1: Regressão Logística
print("\n" + "="*60)
print("🔵 Modelo 1: Regressão Logística")
print("="*60)

lr_model = LogisticRegression(C=1.0, max_iter=100, random_state=42)
lr_params = {'C': 1.0, 'max_iter': 100, 'random_state': 42}
lr_metrics = train_and_evaluate(lr_model, "Logistic_Regression", lr_params)

print("Métricas:")
for name, value in lr_metrics.items():
    print(f"  {name}: {value:.4f}")

In [ ]:
# Modelo 2: Árvore de Decisão
print("\n" + "="*60)
print("🌳 Modelo 2: Árvore de Decisão")
print("="*60)

dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_params = {'max_depth': 5, 'random_state': 42}
dt_metrics = train_and_evaluate(dt_model, "Decision_Tree", dt_params)

print("Métricas:")
for name, value in dt_metrics.items():
    print(f"  {name}: {value:.4f}")

In [ ]:
# Modelo 3: Random Forest
print("\n" + "="*60)
print("🌲 Modelo 3: Random Forest")
print("="*60)

rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_params = {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}
rf_metrics = train_and_evaluate(rf_model, "Random_Forest", rf_params)

print("Métricas:")
for name, value in rf_metrics.items():
    print(f"  {name}: {value:.4f}")

In [ ]:
# Comparar resultados
comparison_df = pd.DataFrame({
    'Logistic Regression': lr_metrics,
    'Decision Tree': dt_metrics,
    'Random Forest': rf_metrics
}).T

print("\n" + "="*60)
print("📊 COMPARAÇÃO DE MODELOS")
print("="*60)
print(comparison_df.to_string())

# Melhor modelo
best_model = comparison_df['f1_score'].idxmax()
print(f"\n🏆 Melhor modelo (F1-Score): {best_model}")

In [ ]:
# Visualizar comparação
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico 1: Todas as métricas
comparison_df.plot(kind='bar', ax=axes[0])
axes[0].set_title('Comparação de Métricas por Modelo')
axes[0].set_ylabel('Score')
axes[0].set_ylim([0, 1])
axes[0].legend(title='Modelo', bbox_to_anchor=(1.05, 1))
axes[0].grid(axis='y', alpha=0.3)

# Gráfico 2: F1-Score
comparison_df['f1_score'].plot(kind='barh', ax=axes[1], color=['blue', 'green', 'orange'])
axes[1].set_title('F1-Score por Modelo')
axes[1].set_xlabel('F1-Score')
axes[1].set_xlim([0, 1])
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

---

## 7. Buscar e Analisar Runs Programaticamente

In [ ]:
# Obter experimento atual
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

print(f"Experimento: {experiment.name}")
print(f"ID: {experiment_id}")

In [ ]:
# Buscar todas as runs do experimento
client = MlflowClient()
runs = client.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.f1_score DESC"]
)

print(f"\nTotal de runs encontradas: {len(runs)}\n")

# Exibir top 5 runs
print("🏆 Top 5 Runs (ordenado por F1-Score):")
print("="*80)

for i, run in enumerate(runs[:5], 1):
    run_name = run.data.tags.get('mlflow.runName', 'N/A')
    f1 = run.data.metrics.get('f1_score', 0)
    accuracy = run.data.metrics.get('accuracy', 0)
    
    print(f"{i}. {run_name}")
    print(f"   F1-Score: {f1:.4f} | Accuracy: {accuracy:.4f}")
    print(f"   Run ID: {run.info.run_id}")
    print()

In [ ]:
# Converter runs para DataFrame para análise
runs_data = []
for run in runs:
    data = {
        'run_id': run.info.run_id,
        'run_name': run.data.tags.get('mlflow.runName', 'N/A'),
        'model_type': run.data.tags.get('model_type', 'N/A'),
        **run.data.metrics
    }
    runs_data.append(data)

runs_df = pd.DataFrame(runs_data)
print("\nDataFrame de Runs:")
runs_df[['run_name', 'model_type', 'f1_score', 'accuracy', 'roc_auc']].head(10)

---

## 8. Model Registry (Opcional)

Registre o melhor modelo no Model Registry.

In [ ]:
# Nome do modelo no registry
model_name = "churn_predictor_notebook"

# Treinar e registrar melhor modelo
with mlflow.start_run(run_name="best_model_for_registry"):
    
    # Usar Random Forest (melhor modelo)
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_train, y_train)
    
    # Avaliar
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_proba)
    }
    
    # Logar
    mlflow.log_params({'n_estimators': 100, 'max_depth': 10})
    mlflow.log_metrics(metrics)
    
    # Registrar no Model Registry
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name=model_name
    )
    
    print(f"✅ Modelo registrado no Model Registry: {model_name}")
    print(f"\nMétricas:")
    for name, value in metrics.items():
        print(f"  {name}: {value:.4f}")

In [ ]:
# Listar versões do modelo
import time
time.sleep(2)  # Aguardar registro completar

try:
    versions = client.search_model_versions(f"name='{model_name}'")
    
    print(f"\nVersões do modelo '{model_name}':")
    print("="*60)
    
    for version in versions:
        print(f"Versão: {version.version}")
        print(f"  Stage: {version.current_stage}")
        print(f"  Run ID: {version.run_id}")
        print()
except Exception as e:
    print(f"Modelo ainda não registrado ou erro: {e}")

---

## 9. Carregar Modelo do MLFlow

Demonstre como carregar e usar um modelo salvo.

In [ ]:
# Obter Run ID do melhor modelo
best_run = runs[0]  # Primeiro run (ordenado por f1_score DESC)
best_run_id = best_run.info.run_id

print(f"Melhor Run ID: {best_run_id}")
print(f"F1-Score: {best_run.data.metrics['f1_score']:.4f}")

In [ ]:
# Carregar modelo
model_uri = f"runs:/{best_run_id}/model"
loaded_model = mlflow.sklearn.load_model(model_uri)

print(f"✅ Modelo carregado de: {model_uri}")
print(f"Tipo: {type(loaded_model)}")

In [ ]:
# Fazer predições com modelo carregado
sample_data = X_test.head(10)
predictions = loaded_model.predict(sample_data)
probabilities = loaded_model.predict_proba(sample_data)[:, 1]

# Exibir resultados
results = sample_data.copy()
results['Real'] = y_test.head(10).values
results['Predição'] = predictions
results['Prob. Churn'] = probabilities.round(3)

print("\n📊 Predições de Exemplo:")
print(results[['idade', 'tempo_cliente_meses', 'satisfacao', 'Real', 'Predição', 'Prob. Churn']])

---

## 10. Exercícios Práticos

### Exercício 1: Testar Diferentes Hiperparâmetros

Teste diferentes valores de `C` para Regressão Logística:

In [ ]:
# Seu código aqui
# Dica: Use um loop para testar C = [0.1, 0.5, 1.0, 5.0, 10.0]
# e logue cada run no MLFlow

# Exemplo de estrutura:
# for C_value in [0.1, 0.5, 1.0, 5.0, 10.0]:
#     with mlflow.start_run(run_name=f"LR_C_{C_value}"):
#         # Treinar modelo
#         # Logar parâmetros e métricas
#         pass

### Exercício 2: Adicionar Nova Métrica

Calcule e logue a métrica de **especificidade**:

$$\text{Specificity} = \frac{TN}{TN + FP}$$

In [ ]:
# Seu código aqui
# Dica: Use confusion_matrix para obter TN e FP
# from sklearn.metrics import confusion_matrix
# tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
# specificity = tn / (tn + fp)

### Exercício 3: Buscar Runs com Filtro

Busque apenas runs com `accuracy > 0.75`:

In [ ]:
# Seu código aqui
# Dica: Use client.search_runs() com filter_string
# filtered_runs = client.search_runs(
#     experiment_ids=[experiment_id],
#     filter_string="metrics.accuracy > 0.75"
# )

---

## 11. Resumo e Próximos Passos

### O que você aprendeu:

✅ Conceitos básicos do MLFlow (Experiments, Runs, Parameters, Metrics)
✅ Como fazer tracking de modelos de ML
✅ Como logar artefatos (gráficos, modelos)
✅ Como comparar múltiplos experimentos
✅ Como usar o Model Registry
✅ Como carregar modelos salvos
✅ Como analisar runs programaticamente

### Próximos Passos:

1. **Explore a UI do MLFlow**: Execute `mlflow ui` e visualize todos os experimentos
2. **Execute os scripts Python**: Teste os exemplos em `models/01_*.py`
3. **Leia a documentação**: Veja `tutorial/` para guias detalhados
4. **Experimente**: Teste diferentes modelos e hiperparâmetros
5. **Integre em seus projetos**: Use MLFlow em seus próprios projetos de ML

---

### Recursos Adicionais:

- [Documentação oficial do MLFlow](https://mlflow.org/docs/latest/index.html)
- [MLFlow GitHub](https://github.com/mlflow/mlflow)
- [Tutorial completo em `tutorial/`](../tutorial/)

---

**🎉 Parabéns por completar o tutorial!**